In [ ]:
import random
from copy import deepcopy


# STRUTTURA DI UN NODO: (Matrice, f(n), g(n), NodoPadre)
MATRICE = 0
F = 1
G = 2
PADRE = 3

DIM_MATRICE = 3 # la matrice è sempre quadrata
BlankSpace = 0 # valore che considero come spazio vuoto

def GenerateGoalMatrix():
    OrderedPosition = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    GoalMatrix = []

    for i in range(DIM_MATRICE):
        NewRow = []

        for j in range(DIM_MATRICE):
            NewRow.append(OrderedPosition[(i*DIM_MATRICE)+j])
        
        GoalMatrix.append(NewRow)

    return GoalMatrix

def RandomizeState():
    PossibleValues = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    RandomizedList = []

    for _ in range(DIM_MATRICE):
        NewRow = []

        for _ in range(DIM_MATRICE):
            RandomElementPosition = random.randint(0,len(PossibleValues)-1)
            NewRow.append(PossibleValues.pop(RandomElementPosition))
        
        RandomizedList.append(NewRow)
    
    return RandomizedList

def GetPositionFromState(state,num):
    IndexRow = IndexCol = -1
    i = j = 0
    FoundNum = False

    while i < DIM_MATRICE and not FoundNum:
        while j < DIM_MATRICE and not FoundNum:
            if state[i][j] == num:
                IndexRow = i
                IndexCol = j
                FoundNum = True

            j+=1
        j=0
        i+=1
    
    return IndexRow,IndexCol

def PrintPosition(state):
    for i in range(DIM_MATRICE): 
        for j in range(DIM_MATRICE): 
            print(f" {state[i][j]} ",end="")
        print("")

def IsSolvable(state):
    flat = [cell for row in state for cell in row if cell != 0]
        
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
    
    return inversions % 2 == 0

GoalState = GenerateGoalMatrix()
StartState = RandomizeState()
while not IsSolvable(StartState):
    StartState = RandomizeState()
    
print("Stato iniziale:\n")
PrintPosition(StartState)

Stato iniziale:

 6  0  3 
 1  2  4 
 7  5  8 


In [49]:
# Muovere un tassello può essere visto come muovere lo spazio vuoto;
def ExpandNodes(CurrentState):
    Queue = []
    IndexRow, IndexCol = GetPositionFromState(CurrentState,BlankSpace)

    if IndexCol+1 < DIM_MATRICE:
        MoveRight = deepcopy(CurrentState)

        MoveRight[IndexRow][IndexCol] = MoveRight[IndexRow][IndexCol+1]
        MoveRight[IndexRow][IndexCol+1] = BlankSpace
        Queue.append(MoveRight)

    if IndexCol-1 >= 0:
        MoveLeft = deepcopy(CurrentState)

        MoveLeft[IndexRow][IndexCol] = MoveLeft[IndexRow][IndexCol-1]
        MoveLeft[IndexRow][IndexCol-1] = BlankSpace
        Queue.append(MoveLeft)

    if IndexRow+1 < DIM_MATRICE:
        MoveUp = deepcopy(CurrentState)

        MoveUp[IndexRow][IndexCol] = MoveUp[IndexRow+1][IndexCol]
        MoveUp[IndexRow+1][IndexCol] = BlankSpace
        Queue.append(MoveUp)
    
    if IndexRow-1 >= 0:
        MoveDown = deepcopy(CurrentState)

        MoveDown[IndexRow][IndexCol] = MoveDown[IndexRow-1][IndexCol]
        MoveDown[IndexRow-1][IndexCol] = BlankSpace
        Queue.append(MoveDown)
    
    return Queue

def SearchLowest(StateList):
    lowest = float("inf")
    LowestNodeIndex = -1
    i = 0

    while i < len(StateList):

        if StateList[i][F] < lowest:
            lowest = StateList[i][F]
            LowestNodeIndex = i
        
        i+=1

    return LowestNodeIndex

def PrintPath(Node,Step):
    
    if Node[PADRE] == None:
        print(f"\nStep n.{Step}:")
        PrintPosition(Node[MATRICE])
        return 

    PrintPath(Node[PADRE],Step-1)
    print(f"\nStep n.{Step}:")
    PrintPosition(Node[MATRICE])

In [50]:
# Assumo che il problema rilassato consista nel poter muovere i pezzi ovunque voglia.
# Quindi, la soluzione ottima consiste nel muovere ogni turno un pezzo nella posizione corretta
# e sarà quindi pari al numero di pezzi fuori posto.
def SolveRelaxedProblem(CurrentState):
    OrderedPosition = list(range(1,DIM_MATRICE**2)) + [BlankSpace]
    MismatchedNumbers = 0

    for i in range(DIM_MATRICE):
        for j in range(DIM_MATRICE):
            if CurrentState[i][j] == BlankSpace: continue
            if CurrentState[i][j] != OrderedPosition[(i*3)+j]:
                MismatchedNumbers += 1

    return MismatchedNumbers

def A_star(StartState):
    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0,None))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[MATRICE] in visited: continue
        visited.append(CurrentNode[MATRICE])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[MATRICE])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[MATRICE])}")
        print(f"f(n) = {CurrentNode[F]}")
        print(f"Depth: {CurrentNode[G]}\n")

        if CurrentNode[0] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[MATRICE])
        for Node in NewNodes:
            g = CurrentNode[G]+1
            f = g + SolveRelaxedProblem(Node)
            queue.append((Node,f,g,CurrentNode))
        
        CurrentStep += 1

    return "failure"

FinalState = A_star(StartState)
PrintPath(FinalState,FinalState[G])

Step n.0, posizione attuale:
 6  0  3 
 1  2  4 
 7  5  8 
Valutazione della posizione: 6
f(n) = 6
Depth: 0

Step n.1, posizione attuale:
 6  2  3 
 1  0  4 
 7  5  8 
Valutazione della posizione: 5
f(n) = 6
Depth: 1

Step n.2, posizione attuale:
 6  2  3 
 1  5  4 
 7  0  8 
Valutazione della posizione: 4
f(n) = 6
Depth: 2

Step n.3, posizione attuale:
 6  2  3 
 1  5  4 
 7  8  0 
Valutazione della posizione: 3
f(n) = 6
Depth: 3

Step n.4, posizione attuale:
 0  6  3 
 1  2  4 
 7  5  8 
Valutazione della posizione: 6
f(n) = 7
Depth: 1

Step n.5, posizione attuale:
 6  2  3 
 1  4  0 
 7  5  8 
Valutazione della posizione: 5
f(n) = 7
Depth: 2

Step n.6, posizione attuale:
 6  2  3 
 0  1  4 
 7  5  8 
Valutazione della posizione: 5
f(n) = 7
Depth: 2

Step n.7, posizione attuale:
 6  2  3 
 1  5  0 
 7  8  4 
Valutazione della posizione: 3
f(n) = 7
Depth: 4

Step n.8, posizione attuale:
 1  6  3 
 0  2  4 
 7  5  8 
Valutazione della posizione: 5
f(n) = 7
Depth: 2

Step n.9, posizione

In [ ]:
# In questo caso, assumo che il problema rilassato mi permetta di muovere i pezzi
# anche in spazi non vuoti; questo vuol dire che, per ogni pezzo, spendo n mosse
# per portarlo sulla riga giusta, ed m mosse per portarlo sulla colonna giusta.
# n = abs(RigaFinale - RigaIniziale), m = abs(ColonnaFinale - ColonnaIniziale)
def SolveRelaxedProblem(CurrentState):
    GridCharacter = list(range(1,DIM_MATRICE**2))
    ManhattanDist = 0

    for num in GridCharacter:
        PosFinale_x,PosFinale_y = GetPositionFromState(GoalState,num)
        PosIniziale_x,PosIniziale_y = GetPositionFromState(CurrentState,num)
        
        ManhattanDist += abs(PosFinale_x-PosIniziale_x) + abs(PosFinale_y-PosIniziale_y)
    
    return ManhattanDist

def A_star(StartState):
    queue = []
    visited = []
    CurrentStep = 0

    queue.append((StartState,SolveRelaxedProblem(StartState),0,None))

    while len(queue) != 0:
        index = SearchLowest(queue)
        CurrentNode = queue.pop(index)

        if CurrentNode[MATRICE] in visited: continue
        visited.append(CurrentNode[MATRICE])

        print(f"Step n.{CurrentStep}, posizione attuale:")
        PrintPosition(CurrentNode[MATRICE])
        print(f"Valutazione della posizione: {SolveRelaxedProblem(CurrentNode[MATRICE])}")
        print(f"f(n) = {CurrentNode[F]}")
        print(f"Depth: {CurrentNode[G]}\n")

        if CurrentNode[MATRICE] == GoalState:
            return CurrentNode
        
        NewNodes = ExpandNodes(CurrentNode[MATRICE])
        for Node in NewNodes:
            g = CurrentNode[G]+1
            f = g + SolveRelaxedProblem(Node)
            queue.append((Node,f,g,CurrentNode))
        
        CurrentStep += 1

    return "failure"

FinalState = A_star(StartState)
PrintPath(FinalState,FinalState[G])

Step n.0, posizione attuale:
 6  0  3 
 1  2  4 
 7  5  8 
Valutazione della posizione: 9
f(n) = 9
Depth: 0

Step n.1, posizione attuale:
 0  6  3 
 1  2  4 
 7  5  8 
Valutazione della posizione: 8
f(n) = 9
Depth: 1

Step n.2, posizione attuale:
 6  2  3 
 1  0  4 
 7  5  8 
Valutazione della posizione: 8
f(n) = 9
Depth: 1

Step n.3, posizione attuale:
 1  6  3 
 0  2  4 
 7  5  8 
Valutazione della posizione: 7
f(n) = 9
Depth: 2

Step n.4, posizione attuale:
 6  2  3 
 1  4  0 
 7  5  8 
Valutazione della posizione: 7
f(n) = 9
Depth: 2

Step n.5, posizione attuale:
 6  2  3 
 1  5  4 
 7  0  8 
Valutazione della posizione: 7
f(n) = 9
Depth: 2

Step n.6, posizione attuale:
 6  2  3 
 1  5  4 
 7  8  0 
Valutazione della posizione: 6
f(n) = 9
Depth: 3

Step n.7, posizione attuale:
 6  3  0 
 1  2  4 
 7  5  8 
Valutazione della posizione: 10
f(n) = 11
Depth: 1

Step n.8, posizione attuale:
 6  2  3 
 0  1  4 
 7  5  8 
Valutazione della posizione: 9
f(n) = 11
Depth: 2

Step n.9, posizi